In [ ]:
def extract_first_page_img(file):
    
def extract_text(img):
    
def similarity_cal(text1,text2):
    
def find_duplicate_images(folder_path):

In [1]:
import fitz # PyMuPDF library
from PIL import Image


def extract_first_page_img(file):

    # Open PDF file
    with fitz.open(file) as pdf:
        # Load the first page
        page = pdf.load_page(0)

        # Convert the page to a pixmap object
        pix = page.get_pixmap()

        # Convert the pixmap object to a PIL Image object
        pil_img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

        # Save the image to file (optional)
        #pil_img.save('imagescs.png')
        return pil_img
def extract_text(img):

In [12]:
#!pip install python-Levenshtein==0.12.0



In [11]:
import Levenshtein

def check_similarity(text1, text2):
    distance = Levenshtein.distance(text1, text2)
    similarity = 1 - (distance / max(len(text1), len(text2)))
    return similarity

In [ ]:
import os
# Define the function to find duplicate images in a folder
threshold=0.9

def find_duplicate_images(folder_path):
    # Create a set to store the paths of duplicate images
    set_of_paths = set()
    # Traverse the folder and check for duplicate images
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for i in range(len(filenames)):
           # 0 10
            filename1 = filenames[i]
            filepath1 = os.path.join(dirpath, filename1)
            img1= extract_first_page_img(filepath1)
            text1= extract_text(img1)
            

            for j in range(i+1, len(filenames)):
            #  0  1 10
                filename2 = filenames[j]
                filepath2 = os.path.join(dirpath, filename2)
                img2= extract_first_page_img(filepath2)
                text2= extract_text(img2)
                

                # Predict the similarity of the two texts
                similarity =check_similarity(text1, text2)

                if similarity > threshold:
                    # Duplicate image found
                    print(f'Duplicate image found: {filename1} and {filename2} with similarity score {similarity}')
                    set_of_paths.add(filepath2)
                    
                    
    return list(set_of_paths)

# Call the function to find duplicate images in a folder
find_duplicate_images('path/to/folder')


In [ ]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential

# Define the function to load and preprocess an image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

# Define the function to create a Siamese network
def create_siamese_network(input_shape):
    # Define the two input images
    input_1 = Input(input_shape)
    input_2 = Input(input_shape)

    # Use the VGG16 model as the base model
    base_model = VGG16(weights='imagenet', include_top=False)

    # Define the two branches of the Siamese network
    branch_1 = base_model(input_1)
    branch_2 = base_model(input_2)

    # Define the similarity function
    def euclidean_distance(vectors):
        vector1, vector2 = vectors
        sum_squared = K.sum(K.square(vector1 - vector2), axis=1, keepdims=True)
        return K.sqrt(K.maximum(sum_squared, K.epsilon()))

    # Add a lambda layer to compute the similarity
    similarity_layer = Lambda(euclidean_distance)([branch_1, branch_2])

    # Define the output of the model
    model = Model(inputs=[input_1, input_2], outputs=similarity_layer)

    return model

# Define the function to check for duplicate images in a folder
def check_duplicate_images(folder_path):
    # Load the trained Siamese network
    siamese_model = create_siamese_network((224, 224, 3))
    siamese_model.load_weights('path/to/siamese_network_weights.h5')

    # Create an image data generator
    datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

    # Traverse the folder and check for duplicate images
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for i in range(len(filenames)):
           # 0 10
            filename1 = filenames[i]
            filepath1 = os.path.join(dirpath, filename1)
            img1 = preprocess_image(filepath1)

            for j in range(i+1, len(filenames)):
            #  0  1 10
                filename2 = filenames[j]
                filepath2 = os.path.join(dirpath, filename2)
                img2 = preprocess_image(filepath2)

                # Predict the similarity of the two images
                similarity = siamese_model.predict([img1, img2])[0][0]

                if similarity < 0.1:
                    # Duplicate image found
                    print(f'Duplicate image found: {filename1} and {filename2} with similarity score {similarity}')

# Call the function to check for duplicate images in a folder
check_duplicate_images('path/to/folder')


In [ ]:
1 2 5 4 5 2 7 2 7
a b c d e f g h i

f h e f h i

In [ ]:
import os
import hashlib

# Define the function to compute the hash of an image file
def hash_file(filename):
    # Open the file in binary mode
    with open(filename, 'rb') as f:
        # Read the file contents in chunks
        chunk = f.read(4096)
        # Compute the hash of the file contents
        md5_hash = hashlib.md5()
        while chunk:
            md5_hash.update(chunk)
            chunk = f.read(4096)
        return md5_hash.hexdigest()

# Define the function to find duplicate images in a folder
def find_duplicate_images(folder_path):
    # Create a dictionary to store the hash values and filenames
    hash_dict = {}
    
    # Traverse the folder and compute the hash of each image file
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            if filename.endswith('.jpg') or filename.endswith('.png'):
                filepath = os.path.join(dirpath, filename)
                filehash = hash_file(filepath)
                if filehash in hash_dict:
                    
                    ##if text match >95%
                    # Duplicate image found
                    print(f'Duplicate image found: {filepath} and {hash_dict[filehash]}')
                    
                    
                    #also delete this file
                else:
                    # Add the hash value and filename to the dictionary
                    hash_dict[filehash] = filepath

# Call the function to find duplicate images in a folder
find_duplicate_images('path/to/folder')


In [ ]:
import Levenshtein

def check_similarity(text1, text2, threshold):
    distance = Levenshtein.distance(text1, text2)
    similarity = 1 - (distance / max(len(text1), len(text2)))
    return similarity >= threshold

# Example usage
text1 = "Hello, world!"
text2 = "Hello, world"
threshold = 0.9

if check_similarity(text1, text2, threshold):
    print("The texts are similar.")
else:
    print("The texts are not similar.")